In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import os
import scipy.optimize as opt
from scipy.optimize import LinearConstraint, Bounds

In [3]:
#reading in orlin's cleaned csv data
local_filepath = "/Users/leonlu/Downloads/gold_data.csv" # TODO: I can't import gold_data.csv as a dataframe
gold_data = pd.read_csv(local_filepath, index_col=0)
display(gold_data.head())

,date,AUSHF,dateNoH,ATM_C,OTM_C,ITM_C,ATM_P,OTM_P,ITM_P
0,2020-05-07 01:00:00,378.00,2020-05-07,7.48,5.70,10.10,5.98,4.28,8.24
1,2020-05-08 01:00:00,381.86,2020-05-08,6.90,5.16,8.96,5.32,3.68,7.66
2,2020-05-09 01:00:00,381.68,2020-05-09,6.76,4.80,9.22,5.26,3.56,7.06
3,2020-05-12 01:00:00,380.32,2020-05-12,5.20,3.74,7.66,5.26,3.58,7.86
4,2020-05-13 01:00:00,380.70,2020-05-13,5.10,3.50,7.78,4.22,2.82,6.96


In [14]:
#construct reward matrix R_{i,t} of dimension (K x N)
reward_df = gold_data.iloc[:, -6:].pct_change()
reward_df.dropna(how = 'all', inplace = True)
reward_df.drop(reward_df.index[-1], inplace=True) #drops the last line of data
display(reward_df)

,ATM_C,OTM_C,ITM_C,ATM_P,OTM_P,ITM_P
1,-0.077540,-0.094737,-0.112871,-0.110368,-0.140187,-0.070388
2,-0.020290,-0.069767,0.029018,-0.011278,-0.032609,-0.078329
3,-0.230769,-0.220833,-0.169197,0.000000,0.005618,0.113314
4,-0.019231,-0.064171,0.015666,-0.197719,-0.212291,-0.114504
5,0.313725,0.297143,0.210797,-0.180095,-0.212766,-0.235632
...,...,...,...,...,...,...
507,0.167109,0.153846,0.160896,-0.096463,-0.092827,-0.083951
508,-0.127273,-0.126984,-0.122807,0.149466,0.144186,0.118598
509,-0.059896,-0.050909,-0.016000,-0.241486,-0.292683,-0.166265
510,0.177285,0.168582,0.170732,0.024490,0.097701,-0.052023


### Implementation of Huo and Fu begins here

In [122]:
K = 6 #number of peripheral assets (non AUSHF) in market
gamma_ = 0.95 #confidence level for CVaR performance function F
split_prop = 0.8 #train/test split
N = len(reward_df) #number of rows/timestep data
delta_ = int(split_prop * N) #delta in calculation
lambda_ = 0.5 # risk preference (->1 means UCB1, ->0 means min CVaR)
times_selected = np.zeros(K)
H_matrix = reward_df[:int(split_prop * N)] # temporary definition; change to historical_returns
R_matrix = reward_df[int(split_prop * N):]

# NOTE: Winston (12/06 9:01pm): if you just want to see what happens when you can only trade L < K assets at a time, pick L 
# assets and make a Boolean array whose i-th component is True if you want to trade asset i and is False otherwise. For example,
# if you want to trade assets 1 and 3, then the array is [False, True, False, True, False, False]. Then slice
# H_matrix and R_matrix with that Boolean array along the asset dimension (the columns) so that the dimension only contains the
# assets you want to trade, and proceed as normal.

In [88]:
display(H_matrix)
display(R_matrix)

,ATM_C,OTM_C,ITM_C,ATM_P,OTM_P,ITM_P
1,-0.077540,-0.094737,-0.112871,-0.110368,-0.140187,-0.070388
2,-0.020290,-0.069767,0.029018,-0.011278,-0.032609,-0.078329
3,-0.230769,-0.220833,-0.169197,0.000000,0.005618,0.113314
4,-0.019231,-0.064171,0.015666,-0.197719,-0.212291,-0.114504
5,0.313725,0.297143,0.210797,-0.180095,-0.212766,-0.235632
...,...,...,...,...,...,...
404,-0.192308,-0.346535,-0.122642,-0.092308,-0.166667,-0.036145
405,-0.244898,-0.454545,-0.139785,0.000000,-0.040000,0.020833
406,0.090090,0.333333,0.083333,-0.169492,-0.229167,-0.093878
407,-0.057851,-0.270833,0.015385,-0.326531,-0.513514,-0.126126


,ATM_C,OTM_C,ITM_C,ATM_P,OTM_P,ITM_P
409,8.250000,45.714286,2.141104,4.387500,66.200000,1.316456
410,0.194103,0.174312,0.173828,-0.095128,-0.110119,-0.091075
411,-0.006173,-0.007812,0.019967,-0.133333,-0.150502,-0.128257
412,-0.095238,-0.094488,-0.109299,0.142012,0.188976,0.140230
413,-0.112128,-0.156522,-0.087912,-0.150259,-0.165563,-0.106855
...,...,...,...,...,...,...
507,0.167109,0.153846,0.160896,-0.096463,-0.092827,-0.083951
508,-0.127273,-0.126984,-0.122807,0.149466,0.144186,0.118598
509,-0.059896,-0.050909,-0.016000,-0.241486,-0.292683,-0.166265
510,0.177285,0.168582,0.170732,0.024490,0.097701,-0.052023


In [89]:
# define helper functions used in huo/fu paper
def I(t):
    """
    Definition: Equation 2.1
    """
    if t < K: # strict inequality due to assets being 0, 1, ..., K-1
        selection = t
    else:
        R_bar = R_matrix[:t].mean(axis=0) # take the mean across the time axis
        R_bar = R_bar.to_numpy()
        selection = np.argmax([R_bar[i] + np.sqrt((2 * np.log(t) / times_selected[i])) for i in range(K)])
    times_selected[selection] += 1
    return selection
        
def F(gamma, u, alpha, t):
    """
    Definition: Equation 2.3; estimates the conditional value-at-risk.
    H_matrix: the s-th column of H_matrix (K rows, \delta columns) is the historical returns of the s-th asset
    R_matrix: the s-th column of R_matrix (K rows, t - 1 columns) is the trial of returns observed so far of the s-th asset
    """
    sum1 = np.sum([np.max(-np.dot(H_matrix.iloc[s, :], u) - alpha, 0) for s in range(len(H_matrix))])
    sum2 = np.sum([np.max(-np.dot(R_matrix.iloc[s, :], u) - alpha, 0) for s in range(t-1)])
    return alpha + (sum1 + sum2) / ((delta_ + t - 1) * (1 - gamma))

In [121]:
def omega_C(gamma, t, equal_weights=False):
    """
    Definition: Equation 2.4
    Computes a risk-aware portfolio according to Equation 2.3. 
    t: the time
    """
    def sum_constraint(u):
        return np.sum(u) - 1
    def non_negative_constraint(u):
        return u
    
    alpha = 0.05
    constraints = [{'type': 'eq', 'fun': sum_constraint}, 
                   {'type': 'ineq', 'fun': non_negative_constraint}]
    initial_guess = np.ones(K)/K
    
    if equal_weights:
        return initial_guess
    else:
        result = opt.minimize(
            fun = lambda u: F(gamma, u, alpha, t),
            x0 = initial_guess,
            constraints = constraints
        )

        # NOTE: Winston 12/06 9:04pm: here is some code for optimizing F w.r.t. (u, alpha), where alpha is instead 
        # from a bounded interval. min_price and max_price are the min. and max. prices of any asset in the historical 
        # returns matrix.
        # 
        # The main difference between this code and yours is that F_t takes (u, alpha) as an argument, not just u, 
        # which is what you've implemented.
        # 
        # [(0, 1)] * K are the bounds for the weights; (min_price, max_price) is for alpha
        # nonnegative_bounds = [(0, 1)] * K + [(min_price, max_price)]
        # nonnegative_constraints = lambda x: np.sum(x[:K]) - 1
        #
        # bounds for weights are (-1, 1) since now we allow negative proportions for the weights in u. 
        # absolute_bounds = [(-1, 1)] * K + [(min_price, max_price)]
        # absolute_constraints = lambda x: np.sum(abs(x[:K])) - 1
        # 
        # result = opt.minimize(
        #     F_t, x0 = x0,
        #     constraints = [{'type': 'eq', 'fun': nonnegative_constraints}], # NOTE: can be changed to absolute_constraints
        #     bounds = nonnegative_bounds # NOTE: can be changed to absolute_bounds
        # )
        return result.x

In [147]:
def sequential_selection_algo(K, lmda=lambda_, gamma=gamma_):
    """
    K: number of assets
    gamma: confidence level
    lambda_: risk preference
    """
    returns = []
    for t in range(10):
        # Equation. 2.2: compute omega_M(t)
        omega_M = np.zeros(K) 
        omega_M[I(t)] = 1
        
        # Equation 2.4: compute the risk aware portfolio
        risk_aware_portfolio = omega_C(gamma, t)

        # Equation 2.5: compute convex combination of UCB1 portfolio and risk aware portfolio
        omega_star = lmda * omega_M + (1 - lmda) * risk_aware_portfolio
        # "Receive portfolio reward"
        curr_reward = R_matrix.iloc[t,:].to_numpy()
        returns.append(curr_reward)
    return returns # idk what to return; there might be a formula somewhere

In [148]:
R_matrix.iloc[t,:]

ATM_C    0.176829
OTM_C    0.137500
ITM_C    0.141176
ATM_P   -0.052308
OTM_P   -0.038462
ITM_P   -0.052154
Name: 428, dtype: float64

In [149]:
sequential_selection_algo(K, lambda_, gamma_)

[array([ 8.25      , 45.71428571,  2.14110429,  4.3875    , 66.2       ,
         1.3164557 ]),
 array([ 0.19410319,  0.17431193,  0.17382812, -0.09512761, -0.11011905,
        -0.09107468]),
 array([-0.00617284, -0.0078125 ,  0.01996672, -0.13333333, -0.15050167,
        -0.12825651]),
 array([-0.0952381 , -0.09448819, -0.10929853,  0.14201183,  0.18897638,
         0.14022989]),
 array([-0.11212815, -0.15652174, -0.08791209, -0.15025907, -0.16556291,
        -0.10685484]),
 array([-0.14175258, -0.13745704, -0.11646586,  0.11585366,  0.07539683,
         0.09255079]),
 array([ 0.05405405,  0.04780876,  0.04772727, -0.11202186, -0.12177122,
        -0.09710744]),
 array([-0.00569801,  0.02661597, -0.00650759, -0.03384615, -0.18067227,
        -0.05720824]),
 array([-0.00286533, -0.03703704,  0.00655022,  0.00636943,  0.12307692,
         0.02427184]),
 array([ 0.18965517,  0.15769231,  0.14533623, -0.00949367,  0.05479452,
        -0.01658768])]